In [ ]:
# A jupyter notebook to animate solutions 

In [ ]:



import numpy as np
import torch

from datagen import solve_ocps
from nn import ActorResidualCritic
from optimal_control.ddp import init_ddp,extract_ddp_data
from optimal_control.samples import uniform_samples
from optimal_control.robot import load_robot
from optimal_control.path import exp_path
from optimal_control.viewer import animate_ddps

torch.manual_seed(0)
np.random.seed(0)

##################################################33





In [ ]:
## if Gepetto-gui is not open and running, then uncomment the line below to run the cell

#!gepetto-gui
robot, config = load_robot(display=True)
 

In [ ]:


exp_number  =   1
eps_number  =   20
save_path   =   exp_path(exp_number=exp_number,make_path=False,get_path=True)
model       =   torch.load(save_path+f'/eps_{eps_number}.pth')

In [ ]:
# Let's get an initial starting state
x0s = uniform_samples(nb_samples = 20, robot=robot, config=config)

DDPS_DATA = []


for x0 in x0s:

    #ddp.problem.x0 = x0
    nq      =   robot.model.nq
    q0      =   x0[:nq]
    # Update robot model with initial state
    robot.framesForwardKinematics(q0)
    robot.computeJointJacobians(q0)

    ddp                 =   init_ddp(robot=robot,config=config,x0=x0,actorResidualCritic=model,N_h=model.horizon)
    xs_init, us_init    =   model.warmstart(x0)
    ddp.solve(xs_init,us_init,maxiter=1000,isFeasible=False)
    print(f"Rollouts required to solve this problem : {ddp.iter}")
    ddp_data            =   extract_ddp_data(ddp)
    DDPS_DATA.append(ddp_data) 


In [ ]:
animate_ddps(DDPS_DATA=DDPS_DATA,robot=robot,config=config)